## Machine Learning 프로젝트 수행을 위한 코드 구조화

- ML project를 위해서 사용하는 템플릿 코드를 만듭니다.

1. **필요한 라이브러리와 데이터를 불러옵니다.**


2. **EDA를 수행합니다.** 이 때 EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. **전처리를 수행합니다.** 이 때 중요한건 **feature engineering**을 어떻게 하느냐 입니다.


4. **데이터 분할을 합니다.** 이 때 train data와 test data 간의 분포 차이가 없는지 확인합니다.


5. **학습을 진행합니다.** 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. **hyper-parameter tuning을 수행합니다.** 원하는 목표 성능이 나올 때 까지 진행합니다. 검증 단계를 통해 지속적으로 **overfitting이 되지 않게 주의**하세요.


7. **최종 테스트를 진행합니다.** 데이터 분석 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.

## 1. 라이브러리, 데이터 불러오기

In [197]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm import LGBMClassifier

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

In [198]:
# 데이터를 불러옵니다.
base_path = '../../data/'
# train = pd.read_csv(base_path + 'train.csv', index_col='id')
# test = pd.read_csv(base_path + 'test.csv', index_col='id')
train = pd.read_csv(base_path + 'train_f7.csv', index_col='id')
test = pd.read_csv(base_path + 'test_f7.csv', index_col='id')
submission = pd.read_csv(base_path + 'sample_submission.csv', index_col='id')
print(train.shape, test.shape, submission.shape)

(101763, 25) (67842, 24) (67842, 1)


In [199]:
train['lOBlank_n'] = train['lOBlank_n'].astype('category')
test['lOBlank_n'] = test['lOBlank_n'].astype('category')
train['branchCount_n'] = train['branchCount_n'].astype('category')
test['branchCount_n'] = test['branchCount_n'].astype('category')
train['pure'] = train['pure'].astype('category')
test['pure'] = test['pure'].astype('category')

In [200]:
train = train.drop(columns=['lOBlank_n', 'branchCount_n'])
test = test.drop(columns=['lOBlank_n', 'branchCount_n'])

In [201]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67842 entries, 101763 to 169604
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   loc                67842 non-null  float64 
 1   v(g)               67842 non-null  float64 
 2   ev(g)              67842 non-null  float64 
 3   iv(g)              67842 non-null  float64 
 4   n                  67842 non-null  float64 
 5   v                  67842 non-null  float64 
 6   l                  67842 non-null  float64 
 7   d                  67842 non-null  float64 
 8   i                  67842 non-null  float64 
 9   e                  67842 non-null  float64 
 10  b                  67842 non-null  float64 
 11  t                  67842 non-null  float64 
 12  lOCode             67842 non-null  int64   
 13  lOComment          67842 non-null  int64   
 14  lOBlank            67842 non-null  int64   
 15  locCodeAndComment  67842 non-null  int64   
 16  uni

## 2. EDA

- 데이터에서 찾아야 하는 기초적인 내용들을 확인합니다.


- class imbalance, target distribution, outlier, correlation을 확인합니다.

In [202]:
train.columns

Index(['loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e', 'b', 't',
       'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment', 'uniq_Op',
       'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount', 'defects',
       'pure'],
      dtype='object')

In [203]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101763 entries, 0 to 101762
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   loc                101763 non-null  float64 
 1   v(g)               101763 non-null  float64 
 2   ev(g)              101763 non-null  float64 
 3   iv(g)              101763 non-null  float64 
 4   n                  101763 non-null  float64 
 5   v                  101763 non-null  float64 
 6   l                  101763 non-null  float64 
 7   d                  101763 non-null  float64 
 8   i                  101763 non-null  float64 
 9   e                  101763 non-null  float64 
 10  b                  101763 non-null  float64 
 11  t                  101763 non-null  float64 
 12  lOCode             101763 non-null  int64   
 13  lOComment          101763 non-null  int64   
 14  lOBlank            101763 non-null  int64   
 15  locCodeAndComment  101763 non-null  int

### 3. 전처리

#### 결측치 처리

### 4. 학습 데이터 분할

In [204]:
# 첫번째 테스트용으로 사용하고, 실제 학습시에는 K-Fold CV를 사용합니다.
from sklearn.model_selection import train_test_split

X = train.drop(columns=['defects'])
y = train.defects

# for OOF-prediction split 5% of data as validation dataset.
X_origin_train, X_origin_val, y_origin_train, y_origin_val = train_test_split(X, y, test_size=0.2, random_state=61, stratify=y)

In [205]:
print(X_origin_train.shape, y_origin_train.shape, X_origin_val.shape, y_origin_val.shape)
print(y_origin_train.mean(), y_origin_val.mean())

(81410, 22) (81410,) (20353, 22) (20353,)
0.2266429185603734 0.22664963396059548


### 5. 학습 및 평가

In [206]:
# 간단하게 LightGBM 테스트
# 적당한 hyper-parameter 조합을 두었습니다. (항상 best는 아닙니다. 예시입니다.)
model = LGBMClassifier(
    n_jobs=-1,
    random_state=61
)

In [207]:
print("\nFitting LightGBM...")
model.fit(X_origin_train, y_origin_train)


Fitting LightGBM...


LGBMClassifier(random_state=61)

In [208]:
# metric은 그때마다 맞게 바꿔줘야 합니다.
evaluation_metric = roc_auc_score

In [209]:
print("Prediction")
pred_train = model.predict(X_origin_train)
pred_val = model.predict(X_origin_val)


train_score = evaluation_metric(y_origin_train, pred_train)
val_score = evaluation_metric(y_origin_val, pred_val)

print("Train Score : %.4f" % train_score)
print("Validation Score : %.4f" % val_score)

Prediction
Train Score : 0.6770
Validation Score : 0.6668


### 6. Hyper-parameter Tuning

In [210]:
def optimizer(trial, X, y, K):
    # 조절할 hyper-parameter 조합을 적어줍니다.
    max_depth = trial.suggest_int('max_depth', 15, 25)
    num_leaves = trial.suggest_categorical('num_leaves', [32,64,128,256,512,])
    min_child_samples = trial.suggest_int('min_child_samples', 20, 80)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 0.7)
    n_estimators = trial.suggest_int('n_estimators', 50, 1023)
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.2)

    # 원하는 모델을 지정합니다, optuna는 시간이 오래걸리기 때문에 저는 보통 RF로 일단 테스트를 해본 뒤에 LGBM을 사용합니다.
    model = LGBMClassifier(
        max_depth=max_depth,
        num_leaves=num_leaves,
        min_child_samples=min_child_samples,
        colsample_bytree=colsample_bytree,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        random_state=61,
    )

    # K-Fold Cross validation을 구현합니다.
    folds = StratifiedKFold(n_splits=K, random_state=61, shuffle=True)
    losses = []

    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]

        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]

        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        loss = evaluation_metric(y_val, preds)
        losses.append(loss)

    # K-Fold의 평균 loss값을 돌려줍니다.
    return np.mean(losses)

In [211]:
K = 5   # Kfold 수
opt_func = partial(optimizer, X=X_origin_train, y=y_origin_train, K=K)

study = optuna.create_study(direction="maximize") # 최소/최대 어느 방향의 최적값을 구할 건지.
study.optimize(opt_func, n_trials=50)

[I 2023-10-22 01:18:13,737] A new study created in memory with name: no-name-a2835bb0-aec6-4eca-b9ed-c65742e997b3
[I 2023-10-22 01:21:08,425] Trial 0 finished with value: 0.6597926124849756 and parameters: {'max_depth': 15, 'num_leaves': 512, 'min_child_samples': 62, 'colsample_bytree': 0.6736284279801933, 'n_estimators': 782, 'learning_rate': 0.006246376982764095}. Best is trial 0 with value: 0.6597926124849756.
[I 2023-10-22 01:21:16,979] Trial 1 finished with value: 0.6581391994745622 and parameters: {'max_depth': 16, 'num_leaves': 32, 'min_child_samples': 68, 'colsample_bytree': 0.6274607434814146, 'n_estimators': 206, 'learning_rate': 0.1892416098459412}. Best is trial 0 with value: 0.6597926124849756.
[I 2023-10-22 01:22:03,967] Trial 2 finished with value: 0.6553857967282035 and parameters: {'max_depth': 17, 'num_leaves': 64, 'min_child_samples': 20, 'colsample_bytree': 0.6517142897098358, 'n_estimators': 870, 'learning_rate': 0.09789585354555999}. Best is trial 0 with value: 0.

KeyboardInterrupt: 

In [ ]:
# optuna가 시도했던 모든 실험 관련 데이터
study.trials_dataframe()

In [ ]:
print("Best Score: %.4f" % study.best_value) # best score 출력
print("Best params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들

In [ ]:
# 실험 기록 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

### 7. 테스트 및 제출 파일 생성

In [ ]:
# Make KFold OOF prediction
def oof_preds(best_model, X, y, test):

    # make KFold
    folds = StratifiedKFold(n_splits=K, random_state=61, shuffle=True)
    final_preds = []
    losses = []
    oof = np.full(len(X), np.nan)
    # fitting with best_model
    for i, (train_idx, val_idx) in enumerate(folds.split(X, y)):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]

        print(f"========== Fold {i+1} ==========")
        best_model.fit(X_train, y_train)
        preds = best_model.predict_proba(X_val)[:, 1]
        oof[val_idx] = preds
        test_preds = best_model.predict_proba(test)[:, 1]
        final_preds.append(test_preds)
        loss = evaluation_metric(y_val, preds)

        losses.append(loss)

    avg_loss = np.mean(losses)
    print(f"Loss : {avg_loss:.4f}")
    return final_preds, oof

In [ ]:
test.info() # 결측치 없음.

In [ ]:
## X_test 만들기 : 앞서했던 전처리를 동일하게 적용해주면 됨.


In [ ]:
best_params = study.best_trial.params

# define best model
best_model = LGBMClassifier(**best_params, random_state=61)

# model finalization : 가장 일반적으로 좋은 예측 성능을 냈던 모델로, 전체 데이터 트레이닝.

preds, oof = oof_preds(best_model, X, y, test)
preds = np.mean(preds, axis=0)
preds # 0.7913

In [ ]:
submission['defects'] = preds

In [ ]:
submission.to_csv(base_path+'preds/lightgbm.csv')

In [ ]:
oof_df = pd.DataFrame({
    'lightgbm_oof': oof
})
oof_df.head()

In [ ]:
oof_df.to_csv(base_path+'oof/lightgbm.csv',index_label='id', header=['lightgbm_oof'])

In [ ]:
pd.DataFrame({'importance':best_model.feature_importances_,'column':test.columns}).sort_values(by='importance', ascending=False)